In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tensorflow.keras.preprocessing.text import Tokenizer, text_to_word_sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Embedding
from tensorflow.keras.utils import to_categorical
import koreanize_matplotlib
from konlpy.tag import Okt

2024-09-13 17:08:57.494306: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 17:08:58.530047: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdirectml.d6f03b303ac3c4f2eeb8ca631688c9757b361310.so
2024-09-13 17:08:58.530178: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libdxcore.so
2024-09-13 17:08:58.534111: I tensorflow/c/logging.cc:34] Successfully opened dynamic library libd3d12.so
2024-09-13 17:08:58.723947: I tensorflow/c/logging.cc:34] DirectML device enumeration: found 1 compatible adapters.


In [2]:
train_data = pd.read_csv("https://raw.githubusercontent.com/haram4th/data4mdai/main/hotelscom_review_train.csv")
test_data = pd.read_csv('./data/hotelscom_review_test.csv')

In [3]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53964 entries, 0 to 53963
Data columns (total 2 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   description  53964 non-null  object
 1   isgood       53964 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 843.3+ KB


# 토큰화

In [4]:
docs = train_data['description']
y = train_data['isgood']

In [5]:
okt = Okt()
okt.morphs(docs[0])

['사람',
 '이',
 '너무',
 '많고',
 '작은',
 '수영장',
 '과',
 '조식',
 '수',
 '용한',
 '계',
 '로',
 '모두',
 '포기',
 '하고',
 '옆',
 '의',
 '아이파크몰',
 '에서',
 '그냥',
 '식사',
 '함']

In [6]:
# # 전체 문장을 토큰화 후 tokenized_docs에 저장
# tokenized_docs = docs.apply(okt.morphs)
# tokenized_docs

In [7]:
import joblib
# joblib.dump(tokenized_docs, './model/hotels_review_token.joblib')

In [8]:
tokenized_docs = joblib.load('./model/hotels_review_token.joblib')

In [9]:
# 단어 인덱스 생성
token = Tokenizer(lower = False)
token.fit_on_texts(tokenized_docs)
print(len(token.word_index))

63571


In [10]:
# 문장 벡터화
X = token.texts_to_sequences(tokenized_docs)
print(X[0])

[147, 1, 10, 362, 381, 124, 24, 22, 39, 8795, 2621, 20, 126, 2252, 36, 179, 13, 3939, 12, 145, 258, 190]


In [11]:
# 가장 긴 문장의 길이 구하기
max_len = max(len(i) for i in X)
max_len

554

In [12]:
# 패딩 주기
# 임베딩을 하면 max_len에 1을 더하지 않아도 됨.
X_padded = pad_sequences(X, maxlen = max_len, padding = 'post')

In [13]:
# 홀드아웃
# 데이터 나누기
from sklearn.model_selection import train_test_split

In [14]:
X_train, X_valid, y_train, y_valid = train_test_split(X_padded, y, test_size = 0.3, stratify = y, random_state = 7)

In [15]:
# 임베딩에 입력될 단어 수
word_size = len(token.word_index) + 1
print(word_size)

63572


## 양방향 RNN 네트워크를 이용해 텍스트 분석

In [16]:
from tensorflow.keras.layers import Embedding, SimpleRNN, Dense, Dropout, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [17]:
birnn = Sequential()
birnn.add(Embedding(word_size, 64, input_length = max_len))
birnn.add(Bidirectional(SimpleRNN(128, return_sequences = True, activation = 'tanh')))
birnn.add(Dropout(0.5))
birnn.add(SimpleRNN(64, activation = 'tanh'))
birnn.add(Dropout(0.5))
birnn.add(Dense(32, activation = 'relu'))
birnn.add(Dense(1, activation = 'sigmoid'))
birnn.summary()

2024-09-13 17:09:17.734767: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-09-13 17:09:17.736435: I tensorflow/c/logging.cc:34] DirectML: creating device on adapter 0 (AMD Radeon(TM) Graphics)
2024-09-13 17:09:17.818755: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-13 17:09:17.818798: W tensorflow/core/common_runtime/pluggable_device/pluggable_device_bfc_allocator.cc:28] Overriding allow_growth setting because force_memory_growth was requested by the device.
2024-09-13 17:09:17.818823: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 554, 64)           4068608   
                                                                 
 bidirectional (Bidirectiona  (None, 554, 256)         49408     
 l)                                                              
                                                                 
 dropout (Dropout)           (None, 554, 256)          0         
                                                                 
 simple_rnn_1 (SimpleRNN)    (None, 64)                20544     
                                                                 
 dropout_1 (Dropout)         (None, 64)                0         
                                                                 
 dense (Dense)               (None, 32)                2080      
                                                        

In [18]:
birnn.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])
modelpath = './model/hotels_review_birnn.keras'
checkpoint = ModelCheckpoint(filepath = modelpath, save_best_only = True)
earlystop = EarlyStopping(patience = 10)

In [ ]:
birnn_history = birnn.fit(X_train, y_train, epochs = 1000, batch_size = 8,
                         validation_data = (X_valid, y_valid),
                         callbacks = [earlystop, checkpoint])

Epoch 1/1000


2024-09-13 17:09:21.285107: I tensorflow/core/grappler/optimizers/custom_graph_optimizer_registry.cc:114] Plugin optimizer for device_type GPU is enabled.
2024-09-13 17:09:21.748370: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-13 17:09:21.748431: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:272] Created TensorFlow device (/job:localhost/replica:0/task:0/device:GPU:0 with 14853 MB memory) -> physical PluggableDevice (device: 0, name: DML, pci bus id: <undefined>)
2024-09-13 17:09:21.750964: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_factory.cc:306] Could not identify NUMA node of platform GPU ID 0, defaulting to 0. Your kernel may not have been built with NUMA support.
2024-09-13 17:09:21.751024: I tensorflow/core/common_runtime/pluggable_device/pluggable_device_f

  14/4722 [..............................] - ETA: 9:08:24 - loss: 0.5751 - accuracy: 0.7232